In [4]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [14]:
model = mz.models.MatchSRNN()
model.params['task'] = mz.tasks.Ranking()
model.params['channels'] = 4
model.params['units'] = 10
model.params['dropout'] = 0.0
model.params['direction'] = 'lt'
model.guess_and_fill_missing_params(verbose=0)
model.build()

AttributeError: 'SpatialGRU' object has no attribute '_channel'

In [2]:
preprocessor = mz.models.MatchSRNN.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8097.09it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4073.59it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 1996322.22it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9220.14it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 3943.37it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 819445.67it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 8102.88it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f2f840541d0>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f2f840cdef0>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [7]:
padding_callback = mz.models.ArcII.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [8]:
model = mz.models.MatchSRNN()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['channels'] = 4
model.params['units'] = 10
model.params['dropout'] = 0.2
model.params['direction'] = 'lt'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

MatchSRNN(
  (embedding): Embedding(30058, 100)
  (dropout): Dropout(p=0.2, inplace=False)
  (matching_tensor_network): MatchingTensor()
  (spatial_gru_network): SpatialGRU(
    (_activation): Tanh()
    (_recurrent_activation): Sigmoid()
  )
  (out): Linear(in_features=10, out_features=1, bias=True)
)
Trainable params:  3048611


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-102 Loss-0.981]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4732 - normalized_discounted_cumulative_gain@5(0.0): 0.5249 - mean_average_precision(0.0): 0.4993



[Iter-204 Loss-0.722]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5924 - normalized_discounted_cumulative_gain@5(0.0): 0.6497 - mean_average_precision(0.0): 0.6104



[Iter-306 Loss-0.525]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5904 - normalized_discounted_cumulative_gain@5(0.0): 0.6608 - mean_average_precision(0.0): 0.6088



[Iter-408 Loss-0.394]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5866 - normalized_discounted_cumulative_gain@5(0.0): 0.6476 - mean_average_precision(0.0): 0.6032



[Iter-510 Loss-0.298]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5726 - normalized_discounted_cumulative_gain@5(0.0): 0.6435 - mean_average_precision(0.0): 0.5984



[Iter-612 Loss-0.190]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5545 - normalized_discounted_cumulative_gain@5(0.0): 0.6129 - mean_average_precision(0.0): 0.5722



[Iter-714 Loss-0.136]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5437 - normalized_discounted_cumulative_gain@5(0.0): 0.6113 - mean_average_precision(0.0): 0.5737



[Iter-816 Loss-0.106]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5248 - normalized_discounted_cumulative_gain@5(0.0): 0.6004 - mean_average_precision(0.0): 0.5565



[Iter-918 Loss-0.078]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5275 - normalized_discounted_cumulative_gain@5(0.0): 0.5777 - mean_average_precision(0.0): 0.549



[Iter-1020 Loss-0.064]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5331 - normalized_discounted_cumulative_gain@5(0.0): 0.6048 - mean_average_precision(0.0): 0.5587

Cost time: 3735.532222509384s
